In [1]:
import numpy as np
import cv2
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

2024-04-15 17:17:12.294093: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-15 17:17:13.385477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model = keras.models.load_model('asl_model.h5')

background = None
accumulated_weight = 0.5   

ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350

In [3]:
# Get a dataset
# Train a model using the dataset and a test.
# Calculate the accuracy it has, and increase the accuracy.
# Then import another dataset and check the predictive accuracy of the old model, and improve it.
# Save the model.
# Using OpenCV, create a Haar Cascade classfier to detec the hand.
# Import the model created before.
# Open the camera, create a frame, and identify the hand
# Predict the gesture using the model.

In [4]:
def run_avg(frame, accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [5]:
def segment_hand(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)

    _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    else:
        hand_segment_max = max(contours, key=cv2.contourArea)
        
        return (thresholded, hand_segment_max)

In [6]:
cam = cv2.VideoCapture(0)
num_frames = 0

while True:
    ret, frame = cam.read()

    if frame is not None:
        frame_copy = frame.copy()
    else:
        continue
    
        # Get the height and width of the frame
    height, width = frame.shape[:2]

    # Get the ROI from the frame
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]

    # Apply grayscale and blur to ROI
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)
    
    if num_frames < 70:
        run_avg(gray, accumulated_weight)
        if num_frames == 1:
            print("[INFO] please wait! calibrating...")
        elif num_frames == 29:
            print("[INFO] calibration successful!")
    else:
        hand = segment_hand(gray)
        
        if hand is not None:
            # If yes, unpack the thresholded image and
            # segmented region
            (thresholded, segmented) = hand

            # Draw the segmented region and display the frame
            cv2.drawContours(frame_copy, [segmented + (ROI_right, ROI_top)], -1, (0, 0, 255))

            # Resize the thresholded image to fit the model input size
            thresholded = cv2.resize(thresholded, (64, 64))
            thresholded = cv2.cvtColor(thresholded, cv2.COLOR_GRAY2RGB)
            thresholded = np.reshape(thresholded, (1,thresholded.shape[0],thresholded.shape[1],3))

            # Predict the gesture using the pre-trained model
            prediction = model.predict(thresholded)
            print(prediction)
    
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right, ROI_bottom), (255,128,0), 3)
    num_frames += 1
    # Display the frame with segmented hand
    cv2.imshow("Video Feed", frame_copy)

    # Observe the keypress by the user
    keypress = cv2.waitKey(1) & 0xFF

    # If the user pressed "q", then stop looping
    if keypress == ord("q"):
        break

# Free up memory
cam.release()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/sachithrka/.local/lib/python3.10/site-packages/cv2/qt/plugins"


[INFO] please wait! calibrating...
[INFO] calibration successful!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
[[0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 6.735749e-22
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
  1.000000e+00]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 4.8248606e-34
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  1.0000000e+00]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0